In [ ]:
from urllib.request import urlopen
from PIL import Image

# Load an AI-generated image of a puppy playing in the snow
puppy_path = "https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/chapter09/images/puppy.png"
image = Image.open(urlopen(puppy_path)).convert("RGB")
caption = "a puppy playing in the snow"
image

In [ ]:
from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel
model_id = "openai/clip-vit-base-patch32"

clip_tokenizer = CLIPTokenizerFast.from_pretrained(model_id)

clip_processor = CLIPProcessor.from_pretrained(model_id)

model = CLIPModel.from_pretrained(model_id)

In [ ]:
inputs = clip_tokenizer(caption, return_tensors="pt")
inputs

In [ ]:
clip_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

In [ ]:
text_embedding = model.get_text_features(**inputs)
text_embedding.shape

In [ ]:
processed_image = clip_processor(
    text=None, images=image, return_tensors="pt"
)["pixel_values"]
processed_image.shape

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
img = processed_image.squeeze(0)
img = img.permute(*torch.arange(img.ndim - 1, -1, -1))
img = np.einsum("ijk->jik", img)
plt.imshow(img)
plt.axis("off")

In [ ]:
image_embedding = model.get_image_features(processed_image)
image_embedding.shape

In [ ]:
text_embedding /= text_embedding.norm(dim=-1, keepdim=True)
image_embedding /= image_embedding.norm(dim=-1, keepdim=True)

text_embedding = text_embedding.detach().cpu().numpy()
image_embedding = image_embedding.detach().cpu().numpy()
score = text_embedding @ image_embedding.T
score

In [ ]:
car_path = "https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/chapter09/images/car.png"
image = Image.open(urlopen(car_path)).convert("RGB")
image

In [ ]:
inputs = blip_processor(image, return_tensors="pt").to(device, torch.float16)
inputs["pixel_values"].shape
blip_processor.tokenizer

In [ ]:
text = "Her vocalization was remarkably melodic"
token_ids = blip_processor(image, text=text, return_tensor="pt")
token_ids = token_ids.to(device, torch.float16)["input_ids"][0]
tokens = blip_processor.tokenizer.convert_ids_to_tokens(token_ids)
tokens
